In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install langchain langchain-community sentence-transformers faiss-cpu pypdf ollama

In [ ]:
!pip install langchain-huggingface

In [ ]:
import ollama
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
PDF_PATH='Ebook-Agentic-AI.PDF'

loader=PyPDFLoader(PDF_PATH)
docs=loader.load()

print('PDF loaded:',len(docs),'pages')

In [ ]:
splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
chunks=splitter.split_documents(docs)

print('chunks created:',len(chunks))

In [ ]:
embedding=SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')

In [ ]:
db=FAISS.from_documents(chunks,embedding)

print('stored in FAISS')

In [ ]:
query= input('Ask a question:')

docs=db.similarity_search_with_score(query,k=3)
print('\n  Retrieved Chunks   \n')

for i,(doc,score) in enumerate(docs):
    print(f'\nchunk {i+1})| Similarity score:{score:.4f}')
    print(doc.page_content[:700])

context='\n\n'.join([doc.page_content for doc,_ in docs])
prompt=f'''
Answer only using the content below.If answeris not in the text,say:
Not found in document.

Context:
{context}

Question:
{query}

Answer:
'''
response=ollama.chat(model='llama3',messages=[{'role':'user','content':prompt}])
answer=response['message']['content']
avg_score=sum(score for _, score in docs)/len(docs)
confidence=max(0,min(100,(1-avg_score)*100))
print('Answer')
print(answer)
print(f'\n Confidence Score:{confidence:.2f}%')
